In [1]:
import pandas as pd
import re
import string
from textblob import TextBlob
from nltk.tokenize import sent_tokenize
from nltk.tokenize import TreebankWordTokenizer
from nltk.corpus import stopwords
import gensim
import demoji

In [2]:
import glob
import os
import math

In [3]:
os.chdir("C:/Users/Sannath/Desktop/Data Visualization/Assignment-4")

In [4]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [5]:
%%time
#combine all files in the list
merged_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])

merged_csv = merged_csv[(merged_csv['language']=='English') | (merged_csv['language']=='Russian') | (merged_csv['language']=='Italian')]


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\magic.py:187: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\magic.py:187: DtypeWarning: Columns (10,20) have mixed types. Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\magic.py:187: DtypeWarning: Columns (0,15,20) have mixed types. Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)


Wall time: 24.3 s


In [6]:
text = merged_csv[['content','language']]
text_df = text.sort_values(['language'])

In [7]:
text_list = text.values.tolist()

In [8]:
ind_russian = []
ind_english = []
ind_italian = []

for i in range(len(text_list)):
    if text_list[i][1]=='Russian':
        ind_russian.append(i)
    elif text_list[i][1]=='English':
        ind_english.append(i)
    else :
        ind_italian.append(i)


In [9]:
#(1) filtering hyperlinks and 
#(2) remove numbers and convert text into lowercase
#(3) separate emojis from text. create a list of set of emojis, each set represents the emojis in its respective locations tweet 
#(4) Sentiment analysis (polarity & subjectivity in separate lists)
#(5) identifying '#' and creating a list of all hashtags
#(6) removing special characters from string of special characters and meaningful words
#(7) sentence into words
#(8) filtering punctuations
#(9) stop-words for english,russian and italian tweets

In [10]:
%%time
text_h = []
emoji_list = []
sentiment = []
subjectivity = []
hashtags_list = []
all_hashtags = []
c = 0

punch = string.punctuation

stop_words_e = stopwords.words('english')
stop_words_r = stopwords.words('russian')
stop_words_i = stopwords.words('italian')

for i in range(len(text_list)):
    
    if not text_list[i][0] :
        c += 1
        print(type(text_list[i][0]))

    try:
        # (1)
        tweet = re.sub(r'(https?:\/\/)(\s)*(www\.)?(\s)*((\w|\s)+\.)*([\w\-\s]+\/)*([\w\-]+)((\?)?[\w\s]*=\s*[\w\%&]*)*', '', text_list[i][0], flags=re.MULTILINE)

        # (2)
        tweet = re.sub(r'\d+', '', tweet)
        tweet = tweet.lower()

        # (3)
        emoji_set = demoji.findall(tweet)
        emoji_list.append(emoji_set)
        for emoji in emoji_set:
            tweet = tweet.replace(emoji,'')

        # (4) sentiment analysis
        blob = TextBlob(tweet)
        sentiment.append(blob.sentiment.polarity)
        subjectivity.append(blob.sentiment.subjectivity)

        # (5) remove # and store word in list
        hashtags_per_tweet = re.findall('\#(\w*)',tweet)
        hashtags_list.append(hashtags_per_tweet)
        if len(hashtags_per_tweet)>0:
            for hashtag in hashtags_per_tweet:
                all_hashtags.append(hashtag)

        # (6)removing special characters from string of special characters and meaningful words
        tweet = tweet.translate(str.maketrans('', '', string.punctuation))

        # (7)
        tweet1 = TreebankWordTokenizer().tokenize(tweet)
        tweet1.sort()

        # (8)
        tweet1 = [words for words in tweet if words not in punch]

        # (9)
        if i<len(ind_english):
            tweet1 = [words for words in tweet if words not in stop_words_e]
        elif i>=len(ind_english) and i<len(ind_russian):
            tweet1 = [words for words in tweet if words not in stop_words_r]
        else :
            tweet1 = [words for words in tweet if words not in stop_words_i]
        text_h.append(tweet1)
    except:
        if ((math.isnan(text_list[i][0])) | (math.isinf(text_list[i][0]))):
            print('text_list')
            text_list[i][0] = ''
            text_h.append(text_list[i][0])
    


text_list
Wall time: 35min 9s


In [ ]:
#export to csv
# merged_csv.to_csv( "merged_csv_en_rs_it.csv", index=False, encoding='utf-8-sig')

In [3]:
## One hot vector

In [ ]:
all_words = [] 
for word_list in text_h:
    for word in word_list:
        all_words.append(word)
all_unique_words = list(dict.fromkeys(all_words))
all_unique_words.sort()

wordDictA = dict.fromkeys(wordSet, 0)